In [1]:
import pandas as pd
import numpy as np
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
ratings = pd.read_csv("ratings.csv")
ratings_split = pd.DataFrame(ratings["UserId:ItemId"].str.split(':').tolist(),columns = ['UserId','ItemId'])
ratings["UserId"] = ratings_split["UserId"]
ratings["ItemId"] = ratings_split["ItemId"]
ratings.drop(columns="UserId:ItemId", inplace=True)
# ratings.set_index("UserId",inplace=True)
# ratings.head()
ratings.isnull().sum()


Prediction    0
Timestamp     0
UserId        0
ItemId        0
dtype: int64

In [3]:
targets = pd.read_csv("targets.csv")
targets_split = pd.DataFrame(targets["UserId:ItemId"].str.split(':').tolist(),columns=["UserId","ItemId"])
targets["UserId"] = targets_split["UserId"]
targets["ItemId"] = targets_split["ItemId"]
targets.drop(columns="UserId:ItemId", inplace=True)
targets.set_index("UserId",inplace=True)

# targets.head()
targets.isnull().sum()

ItemId    0
dtype: int64

In [4]:
ratings.head()

,Prediction,Timestamp,UserId,ItemId
0,6,1362062307,u0026762,i2171847
1,8,1362062624,u0026502,i0444778
2,6,1362062838,u0004598,i1411238
3,7,1362063503,u0031317,i1496422
4,5,1362063653,u0024257,i0118799


In [5]:
targets.head()

,ItemId
UserId,
u0000039,i0060196
u0000039,i0099077
u0000039,i0102926
u0000039,i0109830
u0000039,i0113277


In [6]:
ratings["count_item_apperances"] = ratings.groupby("ItemId")["ItemId"].transform('count')
ratings["count_user_apperances"] = ratings.groupby("UserId")["UserId"].transform('count')
ratings = ratings.loc[(ratings["count_item_apperances"] > 10) | (ratings["ItemId"].isin(targets["ItemId"].unique().tolist()))]
ratings = ratings.loc[(ratings["count_user_apperances"] > 10) | (ratings["UserId"].isin(targets.index.unique().tolist()))]

In [7]:
len(ratings["ItemId"].unique().tolist())

7266

In [8]:
len(targets["ItemId"].unique().tolist())

9698

In [9]:
ratings

,Prediction,Timestamp,UserId,ItemId,count_item_apperances,count_user_apperances
2,6,1362062838,u0004598,i1411238,43,169
3,7,1362063503,u0031317,i1496422,100,53
4,5,1362063653,u0024257,i0118799,102,176
5,4,1362063879,u0024257,i0338013,209,176
7,8,1362064047,u0024257,i0064665,28,176
...,...,...,...,...,...,...
336666,8,1421527966,u0028976,i2179136,143,25
336667,10,1421528061,u0009522,i1187043,111,20
336668,8,1421528114,u0034255,i0166924,55,49
336670,7,1421528203,u0023373,i2084970,352,72


In [10]:
ratings.set_index("UserId",inplace=True)
matrix = pd.DataFrame(index=pd.Index(ratings.index.unique().tolist()))
matrix


""
u0004598
u0031317
u0024257
u0029372
u0006099
...
u0033172
u0003117
u0012635
u0020391


In [11]:
import time
start_time = time.time()

for item in ratings["ItemId"].unique().tolist():
    predictions = ratings.loc[ratings["ItemId"] == item]["Prediction"]
    matrix[item] = predictions
# your code
time.time() - start_time

135.47189688682556

In [12]:
matrix

,i1411238,i1496422,i0118799,i0338013,i0064665,i0884328,i0949731,i0319343,i1068680,i0376994,...,i2937898,i0068575,i2513092,i0086969,i0165361,i2717822,i0102598,i0884732,i3140100,i3093522
u0004598,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
u0031317,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
u0024257,NaN,NaN,5.0,4.0,8.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
u0029372,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
u0006099,NaN,NaN,NaN,NaN,NaN,NaN,5.0,6.0,7.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
u0033172,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
u0003117,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
u0012635,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
u0020391,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
user_score_mean = matrix.mean(axis=1)
user_score_mean

u0004598    6.270968
u0031317    5.980769
u0024257    5.725806
u0029372    5.209677
u0006099    6.105634
              ...   
u0033172    8.000000
u0003117    8.333333
u0012635    8.000000
u0020391    8.000000
u0026580    5.000000
Length: 7574, dtype: float64

In [16]:
matrix = matrix.apply(lambda x : x - x.mean(),axis=1).fillna(0)
matrix

,i1411238,i1496422,i0118799,i0338013,i0064665,i0884328,i0949731,i0319343,i1068680,i0376994,...,i2937898,i0068575,i2513092,i0086969,i0165361,i2717822,i0102598,i0884732,i3140100,i3093522
u0004598,-0.270968,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
u0031317,0.000000,1.019231,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
u0024257,0.000000,0.000000,-0.725806,-1.725806,2.274194,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
u0029372,0.000000,0.000000,0.000000,0.000000,0.000000,1.790323,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
u0006099,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.105634,-0.105634,0.894366,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
u0033172,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
u0003117,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
u0012635,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
u0020391,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
user_item = matrix.to_numpy()

In [20]:
user_item

array([[-0.27096774,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  1.01923077,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        , -0.72580645, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [21]:
norm = np.reshape(np.linalg.norm(user_item , axis=0), (1,user_item.shape[1]) )
norm

array([[11.04122325, 14.92487564, 13.45074263, ...,  0.87170531,
         0.58247423,  2.07692308]])

In [22]:
norm.shape

(1, 7266)

In [23]:
norm_sum = norm * norm.T # elemem wise multply
norm_sum #divisor da similaridade

array([[121.90861075, 164.78888388, 148.51265225, ...,   9.62469296,
          6.43122797,  22.93177136],
       [164.78888388, 222.75191295, 200.75066113, ...,  13.01009338,
          8.6933554 ,  30.99781864],
       [148.51265225, 200.75066113, 180.92247743, ...,  11.72508381,
          7.83471092,  27.93615778],
       ...,
       [  9.62469296,  13.01009338,  11.72508381, ...,   0.75987015,
          0.50774588,   1.81046488],
       [  6.43122797,   8.6933554 ,   7.83471092, ...,   0.50774588,
          0.33927622,   1.20975416],
       [ 22.93177136,  30.99781864,  27.93615778, ...,   1.81046488,
          1.20975416,   4.31360947]])

In [24]:
item_times_item = user_item.T @ user_item # != m @ m.T
item_times_item


array([[121.90861075,   2.87917146,  -2.82167168, ...,   0.        ,
          0.        ,   0.        ],
       [  2.87917146, 222.75191295,  -1.21804414, ...,   0.        ,
          0.        ,   0.        ],
       [ -2.82167168,  -1.21804414, 180.92247743, ...,   0.        ,
          0.        ,   0.        ],
       ...,
       [  0.        ,   0.        ,   0.        , ...,   0.75987015,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.33927622,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   4.31360947]])

In [25]:
similarity = item_times_item/norm_sum
similarity #matriz de similaridades item X item

array([[ 1.        ,  0.01747188, -0.01899954, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.01747188,  1.        , -0.00606745, ...,  0.        ,
         0.        ,  0.        ],
       [-0.01899954, -0.00606745,  1.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  1.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         1.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  1.        ]])

In [26]:
similarity.shape

(7266, 7266)

In [ ]:
# df = pd.DataFrame(np.random.randint(0,5,size=(4, 5)), columns=list('ABCDE'))
# print (df)
# a=df.to_numpy()
# a*a

In [27]:
item_dict = {}
idx = 0
for item in matrix.columns :
    item_dict[item] = idx
    idx = idx + 1
item_dict

{'i1411238': 0,
 'i1496422': 1,
 'i0118799': 2,
 'i0338013': 3,
 'i0064665': 4,
 'i0884328': 5,
 'i0949731': 6,
 'i0319343': 7,
 'i1068680': 8,
 'i0376994': 9,
 'i1568338': 10,
 'i1127715': 11,
 'i1790885': 12,
 'i0889573': 13,
 'i0265086': 14,
 'i1757746': 15,
 'i0338216': 16,
 'i1371111': 17,
 'i1648179': 18,
 'i1853728': 19,
 'i1351685': 20,
 'i0099685': 21,
 'i1707386': 22,
 'i1606378': 23,
 'i0970179': 24,
 'i0372784': 25,
 'i0468569': 26,
 'i0118843': 27,
 'i0101700': 28,
 'i0112682': 29,
 'i0372824': 30,
 'i0091670': 31,
 'i0087182': 32,
 'i1333125': 33,
 'i0313542': 34,
 'i0825232': 35,
 'i1922777': 36,
 'i1667310': 37,
 'i1308729': 38,
 'i0499549': 39,
 'i0432283': 40,
 'i0875034': 41,
 'i0887912': 42,
 'i1054606': 43,
 'i1055369': 44,
 'i0397892': 45,
 'i1549920': 46,
 'i1010048': 47,
 'i0493464': 48,
 'i0467406': 49,
 'i0145660': 50,
 'i0449088': 51,
 'i0382932': 52,
 'i0408236': 53,
 'i0385752': 54,
 'i0419887': 55,
 'i1907668': 56,
 'i2125666': 57,
 'i1920849': 58,
 'i1772

In [28]:
similarity[item_dict['i1411238']][item_dict['i1496422']]

0.01747187909392153